In [ ]:
!pip install geocoder==1.38.1
!pip install geopandas==0.13.2
!pip install shapely==2.0.2
!pip install keplergl==0.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922315 sha256=5e00a06f83b0659d4a2fe3b65c300acf38ea40cb5474a7dc32a8285155e17275
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [ ]:
import geopandas as gpd
import geocoder
import json
import pandas as pd
from shapely.geometry import LineString, Point, Polygon



In [ ]:
import pandas as pd
import requests
from io import StringIO

def get_api(resource_id, limit=1000, retries=3):
    offset = 0
    api_url = f"https://data.tycg.gov.tw/api/v1/rest/datastore/{resource_id}"
    all_data = pd.DataFrame()

    while True:
        try:
            params = {'limit': limit, 'offset': offset, 'format': 'csv'}
            response = requests.get(api_url, params=params)
            response.raise_for_status()  # 如果狀態碼不是 2xx，將拋出一個異常
            current_data = pd.read_csv(StringIO(response.text))
            # all_data = all_data.append(current_data, ignore_index=True)
            all_data = pd.concat([all_data, current_data], ignore_index=True)


            if len(current_data) < limit:
                break

            # 更新偏移量
            offset += limit
            print("OK")
        except requests.exceptions.RequestException as err:
            print("OOps: Something went wrong", err)

    return all_data

# get_api 函數獲取 API 數據
RID = '3fa9285a-b5f5-4f20-91e2-b628fcc489ac'
api_data = get_api(RID)

if not api_data.empty:
    # DataFrame 保存為本地 CSV 文件
    OUTPUT_FILENAME = 'traffic.csv'
    api_data.to_csv(OUTPUT_FILENAME, index=False)

    print(f"Data saved to {OUTPUT_FILENAME}")
else:
    print("Failed to retrieve API data.")


OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OOps: Something went wrong 504 Server Error: Gateway Time-out for url: https://data.tycg.gov.tw/api/v1/rest/datastore/3fa9285a-b5f5-4f20-91e2-b628fcc489ac?limit=1000&offset=36000&format=csv
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
Data saved to traffic.csv


In [ ]:
print(api_data)
selected_columns = ['發生日期', '發生時間', 'GPS經度', 'GPS緯度', '地址類型名稱', '發生縣市名稱', '發生市區鄉鎮名稱',
                    '死亡人數_2_30日內', '受傷人數', '光線名稱', '當事者順位', '飲酒情形名稱']

# 選取指定欄位
api_data = api_data[selected_columns]

newcolumns = {
    '發生日期': '日期',
    '發生時間': '時間',
    'GPS經度': '經度',
    'GPS緯度': '緯度',
    '地址類型名稱': '道路類型',
    '發生縣市名稱': '縣市',
    '發生市區鄉鎮名稱': '市區鄉鎮',
    '死亡人數_2_30日內': '死亡人數',
    '受傷人數': '受傷人數',
    '光線名稱': '白天or夜晚',
    '當事者順位': '當事者順位',
    '飲酒情形名稱': '是否飲酒'
}

# 使用rename方法更改欄位名稱
api_data.rename(columns=newcolumns, inplace=True)


# 挑選發生市區鄉鎮名稱是中壢區及當事者順位=1的資料
df_loc = api_data[(api_data['市區鄉鎮'] == '中壢區') & (api_data['當事者順位'] == 1)]


# 顯示挑選出來的資料
print(df_loc)

        飲酒情形代碼  當事者行動狀態大類別代碼 車輛撞擊部位子類別名稱_最初 車輛撞擊部位子類別名稱_其他 道路型態大類別名稱 發生交叉路口_段  \
0          2.0           1.0           左側車身                      交岔路      NaN   
1          NaN           NaN                                     交岔路      NaN   
2          2.0           1.0           左側車身                      交岔路            
3          NaN           NaN                                     交岔路            
4         10.0           2.0         非汽(機)車                      交岔路      NaN   
...        ...           ...            ...            ...       ...      ...   
109169     2.0           1.0           左側車身           左側車身       交岔路            
109170     2.0           1.0            前車頭            前車頭       交岔路            
109171    10.0           2.0         非汽(機)車                      交岔路            
109172     2.0           1.0            前車頭                     單路部分            
109173     NaN           NaN                                    單路部分            

       道路障礙_視距名稱  事故類型及型態子類

<ipython-input-8-f6f40a3b323d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  api_data.rename(columns=newcolumns, inplace=True)


In [ ]:
df_loc['日期'] = df_loc['日期'].astype('str')
df_loc['日期'] = pd.to_datetime(df_loc['日期'], format='%Y-%m-%d')
df_loc[['年', '月', '日']] = df_loc['日期'].dt.strftime('%Y-%m-%d').str.split('-', expand=True)

# 將 '年'、'月'、'日' 欄位放在 '日期' 欄位前面
df_loc = df_loc[['年', '月', '日', '日期'] + df_loc.columns.difference(['年', '月', '日', '日期']).tolist()]

print(df_loc)


In [ ]:
# 使用 map 函數
df_loc['白天or夜晚'] = df_loc['白天or夜晚'].map({'夜間(或隧道、地下道、涵洞)有照明': '夜晚', '日間自然光線': '白天','晨或暮光':'白天'})


df_loc.loc[df_loc['是否飲酒'].str.contains('呼氣檢測'), '是否飲酒'] = '是'
df_loc.loc[df_loc['是否飲酒'].str.contains('無酒精反應'), '是否飲酒'] = '否'


# 顯示結果
print(df_loc)

df_loc.to_csv('done.csv', index=False, encoding='UTF-8')
